In [1]:
import os
import time
import pandas as pd
from tqdm import notebook
from datetime import date, datetime

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [2]:
class_video = "style-scope ytd-rich-grid-media"
x_path_body = """//*[@id="primary"]"""
xpath_video = """//*[@id="meta"]"""
xpath_video_title = """//*[@id="video-title"]"""
xpath_views = """//*[@id="metadata-line"]/span[1]"""
xpath_time_posted = """//*[@id="metadata-line"]/span[2]"""

In [3]:
channels = ['munkonggadgetREVIEWS','Mercular']

In [4]:
try:
    driver.quit()
except Exception:
    pass

options = webdriver.ChromeOptions() 
options.add_argument('--headless') 
    
df = None
for channel in notebook.tqdm(channels):
    url = f"https://www.youtube.com/@{channel}/videos"
    driver = webdriver.Chrome(options=options)
    driver.get(url)

    video_titles = driver.find_elements(By.XPATH, xpath_video_title)
    last_n_videos = len(video_titles)
    # scroll down
    while True:
        for i in range(10):
            element = driver.find_element(By.XPATH, "/html/body")
            element.send_keys(Keys.PAGE_DOWN)
            time.sleep(1)
        video_titles = driver.find_elements(By.XPATH, xpath_video_title)
        new_n_videos = len(video_titles)
        if new_n_videos == last_n_videos:
            break
        last_n_videos = new_n_videos
    
    views = driver.find_elements(By.XPATH, xpath_views)
    time_posted = driver.find_elements(By.XPATH, xpath_time_posted)
    video_attributes = {
        'video_title':[x.text for x in video_titles],
        'views':[x.text for x in views],
        'time_posted':[x.text for x in time_posted],
    }
    df_i = pd.DataFrame(video_attributes)
    df_i['channel'] = channel
    df_i['time_scrapped'] = datetime.now()
    df = pd.concat([df, df_i])
    driver.quit()

  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
if not os.path.isdir('./data'):
    os.makedirs('./data')

In [6]:
df.to_parquet(f'./data/gadget_youtube_{date.today()}.parquet')